# Delta Review

There are a few key operations necessary to understand and make use of [Delta Lake](https://docs.delta.io/latest/quick-start.html#create-a-table).

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you will:<br>
- Create a Delta Table
- Read data from your Delta Table
- Update data in your Delta Table
- Access previous versions of your Delta Table using [time travel](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html) 
- [Understand the Transaction Log](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html)

In this notebook we will be using the SF Airbnb rental dataset from [Inside Airbnb](http://insideairbnb.com/get-the-data.html).

###Why Delta Lake?<br><br>

<div style="img align: center; line-height: 0; padding-top: 9px;">
  <img src="https://user-images.githubusercontent.com/20408077/87175470-4d8e1580-c29e-11ea-8f33-0ee14348a2c1.png" width="500"/>
</div>

At a glance, Delta Lake is an open source storage layer that brings both **reliability and performance** to data lakes. Delta Lake provides ACID transactions, scalable metadata handling, and unifies streaming and batch data processing. 

Delta Lake runs on top of your existing data lake and is fully compatible with Apache Spark APIs. [For more information](https://docs.databricks.com/delta/delta-intro.html)

## A brief exploration of files in databricks   
https://docs.databricks.com/files/index.html

In [0]:
import os

In [0]:
#os.listdir("/")
#os.listdir("/dbfs")
os.listdir("/Workspace/")

Out[5]: ['Repos']

### Setting the default database and user name  
##### Substitute "renato" by your name in the `username` variable.

In [0]:
## Put your name here
username = "renato"

dbutils.widgets.text("username", username)
spark.sql(f"CREATE DATABASE IF NOT EXISTS dsacademy_embedded_wave3_{username}")
spark.sql(f"USE dsacademy_embedded_wave3_{username}")
spark.conf.set("spark.sql.shuffle.partitions", 40)

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[6]: DataFrame[key: string, value: string]

### Listing Database names

In [0]:
%sql
SHOW DATABASES;

databaseName
cashman_ho_fi
contact_policy_rbko
customer_lifetime_value_rbro
default
dlt_test
ds_academy_embedded_wave_3
dsa_group2
dsacademy
dsacademy_embedded_wave3_andras
dsacademy_embedded_wave3_anita


In [0]:
display(spark.catalog.listDatabases())  #Alternative

name,catalog,description,locationUri
cashman_ho_fi,spark_catalog,Database for use case cashman-ho-fi.,s3://rbi-apex-at01-ho-aa-non-retail-dev-cashman-ho-fi/cashman_ho_fi
contact_policy_rbko,spark_catalog,Database for use case contact-policy-rbko.,s3://rbi-apex-at01-ho-aa-retail-dev-contact-policy-rbko/contact_policy_rbko
customer_lifetime_value_rbro,spark_catalog,Database for use case customer-lifetime-value-rbro.,s3://rbi-apex-at01-ho-aa-retail-dev-customer-lifetime-value-rbro/customer_lifetime_value_rbro
default,spark_catalog,Default database for databricks clusters.,s3://rbi-apex-at01-databricks-default-database-bucket/default
dlt_test,spark_catalog,,s3://rbi-apex-at01-ho-apex-dev-gdwh-il-exploration-pipeline/gdwh_il_exploration_pipeline/dlt_test.db
ds_academy_embedded_wave_3,spark_catalog,Database for use case ds-academy-embedded-wave-3.,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3
dsa_group2,spark_catalog,,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsa_group2.db
dsacademy,spark_catalog,Database for use case dsacademy.,s3://rbi-apex-at01-ho-demo-dsacademy/dsacademy
dsacademy_embedded_wave3_andras,spark_catalog,,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_andras.db
dsacademy_embedded_wave3_anita,spark_catalog,,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_anita.db


###Creating a Delta Table
First we need to read the [Airbnb dataset](http://insideairbnb.com/get-the-data) as a Spark DataFrame

By default Spark on Databricks works with files on DBFS, until you're explicitly changing the schema.  
But if you want to read a file outside the DBFS using **spark.read** you can use the prefix **file:** followed by the complete path to the file.   
https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader

In [0]:
!pwd

/Workspace/Repos/renato.rocha-souza@rbinternational.com/Embedded_Data_Scientist/Module_B/Day3


In [0]:
!ls -l data

total 8
drwxrwxrwx 2 root root 4096 Oct 20 12:18 airbnb
drwxrwxrwx 2 root root 4096 Oct 20 12:18 mllib


In [0]:
datapath = os.path.join(os.getcwd(), "data", "airbnb", "listings.csv.gz")
datapath = "file://" + datapath
print(datapath)

file:///Workspace/Repos/renato.rocha-souza@rbinternational.com/Embedded_Data_Scientist/Module_B/Day3/data/airbnb/listings.csv.gz


In [0]:
#import pandas as pd
#airbnbDF = pd.read_csv(datapath)
#airbnbDF = spark.createDataFrame(airbnbDF) 

airbnbDF = spark.read.csv(datapath, header="true", inferSchema="true", multiLine="true", escape='"')
airbnbDF.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
15883,https://www.airbnb.com/rooms/15883,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,b&b near Old Danube river,"Four rooms, each one differently and individually designed, really charming with lots of details. The interior: a potpourri of many different styles, completed with precious things brought along from our trips.The spaceOld Danube river, a short walk to one of the small streets here. Everywhere small houses or gardens, a small idyllic island far from hectic city life. Here we settled down a few years ago, built a house for us and the kids and decided: let’s add 4 more rooms. And thus this small b&b was created, anything but standard and Mozart kitsch.Guest accessfree wifi, many books about Vienna, great tipps from the hosts...","small and personalFour rooms at this B&B, each one differently and individually designed, really charming with lots tasteful details. The interior – a potpourri of many different styles – has been enhanced with precious souvenirs from our trips.free parkingYou arrive by car? No worries! Parking is no problem at all! It is possible for free on the street right in front of our house. There are always enough spaces available. Arrive and unload peacefully, and begin your stay without any stress whatsoever! recreation nearbyThe Old Danube River’s beautiful recreational area begins at the end of our street. Kilometers of walks beside the water offer you fun and relaxation without cars and noise. It is a natural treasure in the middle of the city.",https://a0.muscache.com/pictures/18eff738-a737-428d-b653-99f2e79145b9.jpg,62142,https://www.airbnb.com/users/show/62142,Eva,2009-12-11T00:00:00.000+0000,"Vienna, Austria",Mein größtes Hobby: Reisen! Am liebsten mit meinen 3 Kindern um ihnen die weite Welt zu zeigen. Eindrücke von allen möglichen Orten finden so Einfluss im Design vom the rooms. Ich freue mich auf neue Begegnungen!,within a day,50%,33%,f,https://a0.muscache.com/im/pictures/user/2416670c-a785-417e-b345-f74b368678c9.jpg?aki_policy=profile_small,https://a0.muscache.com/im/pictures/user/2416670c-a785-417e-b345-f74b368678c9.jpg?aki_policy=profile_x_medium,Donaustadt,4,6,"['email', 'phone']",t,t,"Vienna, Austria",Donaustadt,null,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,null,1 private bath,1,2,"[""Essentials"", ""Heating"", ""High chair"", ""Hangers"", ""Wifi"", ""Air conditioning"", ""Patio or balcony"", ""Paid parking garage off premises"", ""Luggage dropoff allowed"", ""Shampoo"", ""Long term stays allowed"", ""Pack \u2019n play/Travel crib"", ""Hair dryer"", ""Breakfast"", ""Bed linens"", ""TV"", ""Smoke alarm"", ""Hot water""]",$110.00,1,365,1,1,365,365,1.0,365.0,nul

In [0]:
airbnbDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: timestamp (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: timestamp (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_

The cells below converts the Dataframe to a Delta table using the schema provided by the Spark DataFrame.  
First we need to define a path to where we are going to save the Delta Table

In [0]:
!ls /

BUILD	   bin	       dbfs  home   lib64	 media	proc  sbin  tmp
Users	   boot        dev   lib    libx32	 mnt	root  srv   usr
Workspace  databricks  etc   lib32  local_disk0  opt	run   sys   var


In [0]:
%fs ls /

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,1665944387863
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/git-proxy/,git-proxy/,0,1665944387863
dbfs:/tmp/,tmp/,0,1665944387863


In [0]:
deltaPath = os.path.join("/", "tmp", username)    #We are writing to the root folder and not to the DBFS
if not os.path.exists(deltaPath):
    os.mkdir(deltaPath)
    
print(deltaPath)

/tmp/renato


In [0]:
# Converting Spark DataFrame to Delta Table
dbutils.fs.rm(deltaPath, True)
airbnbDF.write.format("delta").mode("overwrite").save(deltaPath)

You can also create a Delta table in the metastore.

In [0]:
airbnbDF.write.format("delta").mode("overwrite").saveAsTable("deltaReview")

### Checking Table creation in the Default Database

In [0]:
display(spark.catalog.listDatabases())

name,catalog,description,locationUri
cashman_ho_fi,spark_catalog,Database for use case cashman-ho-fi.,s3://rbi-apex-at01-ho-aa-non-retail-dev-cashman-ho-fi/cashman_ho_fi
contact_policy_rbko,spark_catalog,Database for use case contact-policy-rbko.,s3://rbi-apex-at01-ho-aa-retail-dev-contact-policy-rbko/contact_policy_rbko
customer_lifetime_value_rbro,spark_catalog,Database for use case customer-lifetime-value-rbro.,s3://rbi-apex-at01-ho-aa-retail-dev-customer-lifetime-value-rbro/customer_lifetime_value_rbro
default,spark_catalog,Default database for databricks clusters.,s3://rbi-apex-at01-databricks-default-database-bucket/default
dlt_test,spark_catalog,,s3://rbi-apex-at01-ho-apex-dev-gdwh-il-exploration-pipeline/gdwh_il_exploration_pipeline/dlt_test.db
ds_academy_embedded_wave_3,spark_catalog,Database for use case ds-academy-embedded-wave-3.,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3
dsa_group2,spark_catalog,,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsa_group2.db
dsacademy,spark_catalog,Database for use case dsacademy.,s3://rbi-apex-at01-ho-demo-dsacademy/dsacademy
dsacademy_embedded_wave3_andras,spark_catalog,,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_andras.db
dsacademy_embedded_wave3_anita,spark_catalog,,s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_anita.db


In [0]:
%sql
USE dsacademy_embedded_wave3_$username;
SHOW TABLES;

database,tableName,isTemporary
dsacademy_embedded_wave3_renato,data20k,false
dsacademy_embedded_wave3_renato,data40k,false
dsacademy_embedded_wave3_renato,deltareview,false
dsacademy_embedded_wave3_renato,testtable,false


Delta supports partitioning your data using unique values in a specified column.  
Let's partition by the neighborhood column. Partitioning by neighborhood gives us a point of quick comparison between different parts of Vienna.

In [0]:
airbnbDF.write.format("delta").mode("overwrite").partitionBy("neighbourhood_cleansed").option("overwriteSchema", "true").save(deltaPath)

###Understanding the [Transaction Log](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html)
Let's take a look at the Delta Transaction Log. We can see how Delta stores the different neighborhood partitions in separate files. Additionally, we can also see a directory called _delta_log.

In [0]:
display(dbutils.fs.ls(deltaPath))

path,name,size,modificationTime
dbfs:/tmp/renato/_delta_log/,_delta_log/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Alsergrund/,neighbourhood_cleansed=Alsergrund/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Brigittenau/,neighbourhood_cleansed=Brigittenau/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Donaustadt/,neighbourhood_cleansed=Donaustadt/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Dbling/,neighbourhood_cleansed=Dbling/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Favoriten/,neighbourhood_cleansed=Favoriten/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Floridsdorf/,neighbourhood_cleansed=Floridsdorf/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Hernals/,neighbourhood_cleansed=Hernals/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Hietzing/,neighbourhood_cleansed=Hietzing/,0,1666268918174
dbfs:/tmp/renato/neighbourhood_cleansed=Innere Stadt/,neighbourhood_cleansed=Innere Stadt/,0,1666268918174


<div style="img align: center; line-height: 0; padding-top: 9px;">
  <img src="https://user-images.githubusercontent.com/20408077/87174138-609fe600-c29c-11ea-90cc-84df0c1357f1.png" width="500"/>
</div>

When a user creates a Delta Lake table, that table’s transaction log is automatically created in the _delta_log subdirectory. As he or she makes changes to that table, those changes are recorded as ordered, atomic commits in the transaction log. Each commit is written out as a JSON file, starting with 000000.json. Additional changes to the table generate subsequent JSON files in ascending numerical order so that the next commit is written out as 000001.json, the following as 000002.json, and so on.

In [0]:
display(dbutils.fs.ls(deltaPath + "/_delta_log/"))

path,name,size,modificationTime
dbfs:/tmp/renato/_delta_log/00000000000000000000.crc,00000000000000000000.crc,8356,1666268830000
dbfs:/tmp/renato/_delta_log/00000000000000000000.json,00000000000000000000.json,10927,1666268828000
dbfs:/tmp/renato/_delta_log/00000000000000000001.crc,00000000000000000001.crc,8398,1666268908000
dbfs:/tmp/renato/_delta_log/00000000000000000001.json,00000000000000000001.json,89701,1666268906000


Next, let's take a look at a Transaction Log File.

The [four columns](https://docs.databricks.com/delta/delta-utility.html) each represent a different part of the very first commit to the Delta Table, creating the table.
- The add column has statistics about the DataFrame as a whole and individual columns.
- The commitInfo column has useful information about what the operation was (WRITE or READ) and who executed the operation.
- The metaData column contains information about the column schema.
- The protocol version contains information about the minimum Delta version necessary to either write or read to this Delta Table.

In [0]:
display(spark.read.json(deltaPath + "/_delta_log/00000000000000000000.json"))

add commitInfo metaData protocol null List(0818-143730-wmpfo19, Databricks-Runtime/11.1.x-cpu-ml-scala2.12, false, WriteSerializable, List(1339918919883120), WRITE, List(1, 8915373, 11797), List(Overwrite, []), 1666268827254, e6f3a072-3b46-4f2f-abf0-f0f032e84f0f, 5842419088152212, martin.poettler@rbinternational.com) null null null null null List(1, 2) null null List(List(true), 1666268823084, List(parquet), 4791e466-839d-4443-9182-2b3a3a79b792, List(), {"type":"struct","fields":[{"name":"id","type":"long","nullable":true,"metadata":{}},{"name":"listing_url","type":"string","nullable":true,"metadata":{}},{"name":"scrape_id","type":"long","nullable":true,"metadata":{}},{"name":"last_scraped","type":"timestamp","nullable":true,"metadata":{}},{"name":"source","type":"string","nullable":true,"metadata":{}},{"name":"name","type":"string","nullable":true,"metadata":{}},{"name":"description","type":"string","nullable":true,"metadata":{}},{"name":"neighborhood_overview","type":"string","nullable":true,"metadata":{}},{"name":"picture_url","type":"string","nullable":true,"metadata":{}},{"name":"host_id","type":"integer","nullable":true,"metadata":{}},{"name":"host_url","type":"string","nullable":true,"metadata":{}},{"name":"host_name","type":"string","nullable":true,"metadata":{}},{"name":"host_since","type":"timestamp","nullable":true,"metadata":{}},{"name":"host_location","type":"string","nullable":true,"metadata":{}},{"name":"host_about","type":"string","nullable":true,"metadata":{}},{"name":"host_response_time","type":"string","nullable":true,"metadata":{}},{"name":"host_response_rate","type":"string","nullable":true,"metadata":{}},{"name":"host_acceptance_rate","type":"string","nullable":true,"metadata":{}},{"name":"host_is_superhost","type":"string","nullable":true,"metadata":{}},{"name":"host_thumbnail_url","type":"string","nullable":true,"metadata":{}},{"name":"host_picture_url","type":"string","nullable":true,"metadata":{}},{"name":"host_neighbourhood","type":"string","nullable":true,"metadata":{}},{"name":"host_listings_count","type":"integer","nullable":true,"metadata":{}},{"name":"host_total_listings_count","type":"integer","nullable":true,"metadata":{}},{"name":"host_verifications","type":"string","nullable":true,"metadata":{}},{"name":"host_has_profile_pic","type":"string","nullable":true,"metadata":{}},{"name":"host_identity_verified","type":"string","nullable":true,"metadata":{}},{"name":"neighbourhood","type":"string","nullable":true,"metadata":{}},{"name":"neighbourhood_cleansed","type":"string","nullable":true,"metadata":{}},{"name":"neighbourhood_group_cleansed","type":"string","nullable":true,"metadata":{}},{"name":"latitude","type":"double","nullable":true,"metadata":{}},{"name":"longitude","type":"double","nullable":true,"metadata":{}},{"name":"property_type","type":"string","nullable":true,"metadata":{}},{"name":"room_type","type":"string","nullable":true,"metadata":{}},{"name":"accommodates","type":"integer","nullable":true,"metadata":{}},{"name":"bathrooms","type":"string","nullable":true,"metadata":{}},{"name":"bathrooms_text","type":"string","nullable":true,"metadata":{}},{"name":"bedrooms","type":"integer","nullable":true,"metadata":{}},{"name":"beds","type":"integer","nullable":true,"metadata":{}},{"name":"amenities","type":"string","nullable":true,"metadata":{}},{"name":"price","type":"string","nullable":true,"metadata":{}},{"name":"minimum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"maximum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"minimum_minimum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"maximum_minimum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"minimum_maximum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"maximum_maximum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"minimum_nights_avg_ntm","type":"double","nullable":true,"metadata":{}},{"name":"maximum_nights_avg_ntm","type":"do

One key difference between these two transaction logs is the size of the JSON file, this file has 39 rows compared to the previous 4. To understand why, let's take a look at the commitInfo column. We can see that in the operationParameters section, partitionBy has been filled in by the "neighbourhood_cleansed" column. Furthermore, if we look at the add section on row 3, we can see that a new section called partitionValues has appeared. As we saw above, Delta stores partitions separately in memory, however, it stores information about these partitions in the same transaction log file.

In [0]:
display(spark.read.json(deltaPath + "/_delta_log/00000000000000000001.json"))

add commitInfo metaData remove null List(0818-143730-wmpfo19, Databricks-Runtime/11.1.x-cpu-ml-scala2.12, false, WriteSerializable, List(74902913649535), WRITE, List(23, 9513340, 11797), List(Overwrite, ["neighbourhood_cleansed"]), 0, 1666268905813, 708f5ed0-517f-424b-a996-43491b765952, 3892373017852382, renato.rocha-souza@rbinternational.com) null null null null List(List(true), 1666268823084, List(parquet), 4791e466-839d-4443-9182-2b3a3a79b792, List(neighbourhood_cleansed), {"type":"struct","fields":[{"name":"id","type":"long","nullable":true,"metadata":{}},{"name":"listing_url","type":"string","nullable":true,"metadata":{}},{"name":"scrape_id","type":"long","nullable":true,"metadata":{}},{"name":"last_scraped","type":"timestamp","nullable":true,"metadata":{}},{"name":"source","type":"string","nullable":true,"metadata":{}},{"name":"name","type":"string","nullable":true,"metadata":{}},{"name":"description","type":"string","nullable":true,"metadata":{}},{"name":"neighborhood_overview","type":"string","nullable":true,"metadata":{}},{"name":"picture_url","type":"string","nullable":true,"metadata":{}},{"name":"host_id","type":"integer","nullable":true,"metadata":{}},{"name":"host_url","type":"string","nullable":true,"metadata":{}},{"name":"host_name","type":"string","nullable":true,"metadata":{}},{"name":"host_since","type":"timestamp","nullable":true,"metadata":{}},{"name":"host_location","type":"string","nullable":true,"metadata":{}},{"name":"host_about","type":"string","nullable":true,"metadata":{}},{"name":"host_response_time","type":"string","nullable":true,"metadata":{}},{"name":"host_response_rate","type":"string","nullable":true,"metadata":{}},{"name":"host_acceptance_rate","type":"string","nullable":true,"metadata":{}},{"name":"host_is_superhost","type":"string","nullable":true,"metadata":{}},{"name":"host_thumbnail_url","type":"string","nullable":true,"metadata":{}},{"name":"host_picture_url","type":"string","nullable":true,"metadata":{}},{"name":"host_neighbourhood","type":"string","nullable":true,"metadata":{}},{"name":"host_listings_count","type":"integer","nullable":true,"metadata":{}},{"name":"host_total_listings_count","type":"integer","nullable":true,"metadata":{}},{"name":"host_verifications","type":"string","nullable":true,"metadata":{}},{"name":"host_has_profile_pic","type":"string","nullable":true,"metadata":{}},{"name":"host_identity_verified","type":"string","nullable":true,"metadata":{}},{"name":"neighbourhood","type":"string","nullable":true,"metadata":{}},{"name":"neighbourhood_cleansed","type":"string","nullable":true,"metadata":{}},{"name":"neighbourhood_group_cleansed","type":"string","nullable":true,"metadata":{}},{"name":"latitude","type":"double","nullable":true,"metadata":{}},{"name":"longitude","type":"double","nullable":true,"metadata":{}},{"name":"property_type","type":"string","nullable":true,"metadata":{}},{"name":"room_type","type":"string","nullable":true,"metadata":{}},{"name":"accommodates","type":"integer","nullable":true,"metadata":{}},{"name":"bathrooms","type":"string","nullable":true,"metadata":{}},{"name":"bathrooms_text","type":"string","nullable":true,"metadata":{}},{"name":"bedrooms","type":"integer","nullable":true,"metadata":{}},{"name":"beds","type":"integer","nullable":true,"metadata":{}},{"name":"amenities","type":"string","nullable":true,"metadata":{}},{"name":"price","type":"string","nullable":true,"metadata":{}},{"name":"minimum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"maximum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"minimum_minimum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"maximum_minimum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"minimum_maximum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"maximum_maximum_nights","type":"integer","nullable":true,"metadata":{}},{"name":"minimum_nights_avg_ntm","type":"double","nullable":true,"metadata":{}},{"name":"maximum_nig

Finally, let's take a look at the files inside one of the Neighborhood partitions. The file inside corresponds to the partition commit (file 01) in the _delta_log directory.

In [0]:
display(dbutils.fs.ls(deltaPath + "/neighbourhood_cleansed=Donaustadt/"))

path,name,size,modificationTime
dbfs:/tmp/renato/neighbourhood_cleansed=Donaustadt/part-00000-cf646f3c-62e3-4daf-9680-34032f52c9b5.c000.snappy.parquet,part-00000-cf646f3c-62e3-4daf-9680-34032f52c9b5.c000.snappy.parquet,286357,1666268899000


### Reading data from your Delta table

In [0]:
df = spark.read.format("delta").load(deltaPath)
df.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
153988,https://www.airbnb.com/rooms/153988,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,City Center Apartment - Free Wifi unlimited,"Walking distance to City Center! Unlimited WIFI for free!The spaceAmenities: Spacious living room with large sofa bed, sofa, TV and ceiling fan. Hall, bathroom with separate heater, elevator, laundry service, fully equipped kitchen.Excellent transport links / infrastructure: diverse public transportation easy 2 minute walk! Supermarkets, restaurants and pubs nearby! In a few minutes to reach the inner city, many attractions are within walking distance!All of Vienna’s sights are within walking distance. In only a few minutes walk you will come across the largest and longest shopping streets in the city, a real paradise for shopaholics. The city centre and natural and art history museums are practically at your doorstep, while the world famous ‘Staatsoper’ is only a few minutes away. Reaching the public transport network is simple and in no other district in the city are you better connected than in the inner districts. There are a number",null,https://a0.muscache.com/pictures/fd1aed56-e888-473b-858d-8c64d6dc620f.jpg,739130,https://www.airbnb.com/users/show/739130,Roman,2011-06-24T00:00:00.000+0000,"Vienna, Austria",I can't live without hiking in the mountains. I like to travel all over the world and I love South East Asia.,within an hour,100%,100%,f,https://a0.muscache.com/im/users/739130/profile_pic/1377624953/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/739130/profile_pic/1377624953/original.jpg?aki_policy=profile_x_medium,Alsergrund,1,1,"['email', 'phone', 'work_email']",t,t,null,Alsergrund,null,48.21606,16.34875,Entire condo,Entire home/apt,3,null,1 bath,1,2,"[""Dishes and silverware"", ""Cooking basics"", ""Shampoo"", ""Long term stays allowed"", ""Cable TV"", ""Wifi"", ""Elevator"", ""Hangers"", ""Paid dryer \u2013 In building"", ""Stove"", ""Hot water"", ""Hot water kettle"", ""Iron"", ""Essentials"", ""Dishwasher"", ""Free washer \u2013 In building"", ""Oven"", ""Coffee maker"", ""Hair dryer"", ""1\"" TV with standard cable, DVD player"", ""Bed linens"", ""Bathtub"", ""Microwave"", ""Paid parking off premises"", ""Refrigerator"", ""Carbon monoxide alarm"", ""Kitchen"", ""Room-darkening shades"", ""Extra pillows and blankets"", ""Pocket wifi"", ""Host greets you"", ""Smoke alarm"", ""Heating""]",$52.00,1,182,1,5,1125,1125,4.8,1125.0,null,t,5,11,11,233,2022-09-12T00:00:00.000+0000,144,16,2,2011-08-19T00:00:00.000+0000,2022-08-24T00:00:00.000+0000,4.47,4.56,4.7,4.69,4.64,4.66,4.59,null,t,1,1,0,0,1.07
197968,https://www.airbnb.com/rooms/197968,20220911230927,

#Updating your Delta Table

#### Let's filter for rows where the host is a superhost.

In [0]:
df_update = airbnbDF.filter(airbnbDF["host_is_superhost"] == "t")
df_update.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
38768,https://www.airbnb.com/rooms/38768,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,central cityapartement- wifi- nice neighbourhood,"39m² apartment with beautiful courtyard of the house. quiet and safe upcoming neighborhood-located in old jewish quater and popular area of Vienna. Surrounded by nice bars and restaurants, 15 min walkingdistance to the middle of citycenter, close to many sights . 300m to subway U2 Taborstraße. Next to the karmelitermarket (with delicious places to eat) and the Augarten. free wifi. touristtax of 3,2% is included in the price.The spaceHoliday atmosphere apartment, 39m². 2 rooms with free wlan Internet access.Its situated in the heart of Vienna, in a renovated house of the 2nd district, very close to the subwaystation U2 Taborstraße, only 15 min walking distance to St. Stephansdom-cathedral.The Karmelitermarket is around the corner and the neighborhood is with many bars and restaurants is very popular. Some nice restaurants and bars are only about 100 meters walkingdistance.The quiet apartment was recently completely renovated and stylish","the Karmeliterviertel became very popular in the last years. It offers many new pubs and restaurants and is located next to Augarten (huge garden good for jogging), to the concert hall of Wiener Sängerknaben (muth), and has the Karmelitermarket, which is best to visit on Saturday morning to get biological products. In summer the best nightbars are at the donaukanal, which is 5 minutes away.",https://a0.muscache.com/pictures/ad4089a3-5355-4681-96bb-e3ad70684987.jpg,166283,https://www.airbnb.com/users/show/166283,Hannes,2010-07-14T00:00:00.000+0000,"Vienna, Austria","I am open minded and like travelling myself. I have spent many months in Latinamerica and Asia, where I got in touch with Indian philosophie and meditation... Now I mostly work in the field of contemporary art and I do my best to offer you a nice apartment next to the citycenter...!",within an hour,100%,100%,t,https://a0.muscache.com/im/users/166283/profile_pic/1435040494/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/166283/profile_pic/1435040494/original.jpg?aki_policy=profile_x_medium,Leopoldstadt,3,3,"['email', 'phone']",t,t,"Vienna, Austria",Leopoldstadt,null,48.21924,16.37831,Entire rental unit,Entire home/apt,5,null,1 bath,1,3,"[""Dishes and silverware"", ""Cooking basics"", ""Shampoo"", ""Cleaning products"", ""Host greets you"", ""Long term stays allowed"", ""Wifi"", ""Die Mikrowelle hat eine Grillfunktion. Es gibt keinen eigenst\u00e4ndigen Ofen. oven"", ""Hangers"", ""Stove"", ""Hot water"", ""Outdoor dining area"", ""Drying rack for clothing"", ""Hot wate

#### Saving (overwriting) the new filtered dataframe

In [0]:
df_update.write.format("delta").mode("overwrite").save(deltaPath)

#### Reading the new saved table

In [0]:
df = spark.read.format("delta").load(deltaPath)
df.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
197968,https://www.airbnb.com/rooms/197968,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,Central 2 bedroom flat,"I love my flat and want to host only guests who might love it too.I dont consider this a business: It is about hosting people who apreciate my home. I need to know what my potential guests are expecting and if I (my flat) can offer it. Welcome !The space-Brandnew flat- literally walking distance to Stephansplatz (center of the city center) -or with living room -Fully equipped kitchen (stove, dishwasher, microwave oven...)-seperate bathroom and toilet - washmachine and dryer available (in a common room for a small fee)- very close to the AKH (General Hospital)- great for jogging - tram around the corner (7 different lines) - beautiful huge quiet balcony - open kitchen - lots of shops around - large balcony (sun all afternoon) - US Embassy less than 5 minutes walking distance away - University in walking distance - the apartment- complex has an internationally recognised d",Very central but quiet neighborhood.,https://a0.muscache.com/pictures/18345184/eb5fa682_original.jpg,965759,https://www.airbnb.com/users/show/965759,Pierre,2011-08-13T00:00:00.000+0000,"Vienna, Austria",Traveling widens your horizon,within a few hours,100%,97%,t,https://a0.muscache.com/im/pictures/user/fc387e0e-7992-4ea2-affa-138c78c978f1.jpg?aki_policy=profile_small,https://a0.muscache.com/im/pictures/user/fc387e0e-7992-4ea2-affa-138c78c978f1.jpg?aki_policy=profile_x_medium,Alsergrund,2,3,"['email', 'phone']",t,t,"Vienna, Wien, Austria",Alsergrund,null,48.21919,16.35505,Entire rental unit,Entire home/apt,6,null,1 bath,2,4,"[""HDTV with standard cable"", ""Paid parking on premises"", ""Dishes and silverware"", ""Cooking basics"", ""Shampoo"", ""Long term stays allowed"", ""Cable TV"", ""Wifi"", ""Elevator"", ""Hangers"", ""Stove"", ""Hot water"", ""Iron"", ""Washer"", ""Essentials"", ""Dishwasher"", ""Oven"", ""Single level home"", ""Dryer"", ""Coffee maker"", ""Hair dryer"", ""Bed linens"", ""Bathtub"", ""Microwave"", ""Private entrance"", ""Dedicated workspace"", ""Refrigerator"", ""Private patio or balcony"", ""Kitchen"", ""Room-darkening shades"", ""Extra pillows and blankets"", ""Luggage dropoff allowed"", ""Host greets you"", ""Heating""]",$123.00,1,1125,1,1,1125,1125,1.0,1125.0,null,t,30,60,90,365,2022-09-12T00:00:00.000+0000,75,10,1,2012-09-18T00:00:00.000+0000,2022-08-20T00:00:00.000+0000,4.72,4.76,4.73,4.85,4.91,4.77,4.5,null,f,2,2,0,0,0.62
936577,https://www.airbnb.com/rooms/936577,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,LIVE : CENTRAL : AND : NEW !,"The spaceThe apartmenthouse is situated close to Inner City. You

#### Let's look at the files in the "Favoriten" partition post-update.  
#### Remember, the different files in this directory are snapshots of your DataFrame corresponding to different commits.

In [0]:
display(dbutils.fs.ls(deltaPath + "/neighbourhood_cleansed=Favoriten/"))

path,name,size,modificationTime
dbfs:/tmp/renato/neighbourhood_cleansed=Favoriten/part-00000-1250d9a8-95eb-4167-a7f4-2721c400fa4d.c000.snappy.parquet,part-00000-1250d9a8-95eb-4167-a7f4-2721c400fa4d.c000.snappy.parquet,597964,1666268900000
dbfs:/tmp/renato/neighbourhood_cleansed=Favoriten/part-00000-3996de0e-e0e7-48e3-ad58-27facdb07da9.c000.snappy.parquet,part-00000-3996de0e-e0e7-48e3-ad58-27facdb07da9.c000.snappy.parquet,177668,1666269104000


#Delta Time Travel

Let's suppose we have made a mistake and we need the entire dataset back!  
You can access a previous version of your Delta Table using [Delta Time Travel](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html).  

Use the following cell to access your version history.

In [0]:
%sql
DESCRIBE HISTORY delta.`/tmp/renato/`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2022-10-20T12:31:48.000+0000,3892373017852382,renato.rocha-souza@rbinternational.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(74902913649535),0818-143730-wmpfo19,1,WriteSerializable,false,"Map(numFiles -> 23, numOutputRows -> 2637, numOutputBytes -> 2986477)",null,Databricks-Runtime/11.1.x-cpu-ml-scala2.12
1,2022-10-20T12:28:26.000+0000,3892373017852382,renato.rocha-souza@rbinternational.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""neighbourhood_cleansed""])",null,List(74902913649535),0818-143730-wmpfo19,0,WriteSerializable,false,"Map(numFiles -> 23, numOutputRows -> 11797, numOutputBytes -> 9513340)",null,Databricks-Runtime/11.1.x-cpu-ml-scala2.12
0,2022-10-20T12:27:08.000+0000,5842419088152212,martin.poettler@rbinternational.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1339918919883120),0818-143730-wmpfo19,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 11797, numOutputBytes -> 8915373)",null,Databricks-Runtime/11.1.x-cpu-ml-scala2.12


Delta Lake will keep a 30 day version history by default, but if necessary, Delta can store a version history for longer.  
Using the `versionAsOf` option allows you to easily access previous versions of our Delta Table.

In [0]:
df = spark.read.format("delta").option("versionAsOf", 0).load(deltaPath)
df.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
15883,https://www.airbnb.com/rooms/15883,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,b&b near Old Danube river,"Four rooms, each one differently and individually designed, really charming with lots of details. The interior: a potpourri of many different styles, completed with precious things brought along from our trips.The spaceOld Danube river, a short walk to one of the small streets here. Everywhere small houses or gardens, a small idyllic island far from hectic city life. Here we settled down a few years ago, built a house for us and the kids and decided: let’s add 4 more rooms. And thus this small b&b was created, anything but standard and Mozart kitsch.Guest accessfree wifi, many books about Vienna, great tipps from the hosts...","small and personalFour rooms at this B&B, each one differently and individually designed, really charming with lots tasteful details. The interior – a potpourri of many different styles – has been enhanced with precious souvenirs from our trips.free parkingYou arrive by car? No worries! Parking is no problem at all! It is possible for free on the street right in front of our house. There are always enough spaces available. Arrive and unload peacefully, and begin your stay without any stress whatsoever! recreation nearbyThe Old Danube River’s beautiful recreational area begins at the end of our street. Kilometers of walks beside the water offer you fun and relaxation without cars and noise. It is a natural treasure in the middle of the city.",https://a0.muscache.com/pictures/18eff738-a737-428d-b653-99f2e79145b9.jpg,62142,https://www.airbnb.com/users/show/62142,Eva,2009-12-11T00:00:00.000+0000,"Vienna, Austria",Mein größtes Hobby: Reisen! Am liebsten mit meinen 3 Kindern um ihnen die weite Welt zu zeigen. Eindrücke von allen möglichen Orten finden so Einfluss im Design vom the rooms. Ich freue mich auf neue Begegnungen!,within a day,50%,33%,f,https://a0.muscache.com/im/pictures/user/2416670c-a785-417e-b345-f74b368678c9.jpg?aki_policy=profile_small,https://a0.muscache.com/im/pictures/user/2416670c-a785-417e-b345-f74b368678c9.jpg?aki_policy=profile_x_medium,Donaustadt,4,6,"['email', 'phone']",t,t,"Vienna, Austria",Donaustadt,null,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,null,1 private bath,1,2,"[""Essentials"", ""Heating"", ""High chair"", ""Hangers"", ""Wifi"", ""Air conditioning"", ""Patio or balcony"", ""Paid parking garage off premises"", ""Luggage dropoff allowed"", ""Shampoo"", ""Long term stays allowed"", ""Pack \u2019n play/Travel crib"", ""Hair dryer"", ""Breakfast"", ""Bed linens"", ""TV"", ""Smoke alarm"", ""Hot water""]",$110.00,1,365,1,1,365,365,1.0,365.0,nul

You can also access older versions using a timestamp.

**Replace the timestamp string with the information from your version history**. Note that you can use a date without the time information if necessary.

In [0]:
# TO DO WITH YOUR EARLY TIMESTAMPS FROM TWO TABLES ABOVE
timeStampString = "2022-10-20T12:28:26.000+0000"
df = spark.read.format("delta").option("timestampAsOf", timeStampString).load(deltaPath)
df.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
153988,https://www.airbnb.com/rooms/153988,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,City Center Apartment - Free Wifi unlimited,"Walking distance to City Center! Unlimited WIFI for free!The spaceAmenities: Spacious living room with large sofa bed, sofa, TV and ceiling fan. Hall, bathroom with separate heater, elevator, laundry service, fully equipped kitchen.Excellent transport links / infrastructure: diverse public transportation easy 2 minute walk! Supermarkets, restaurants and pubs nearby! In a few minutes to reach the inner city, many attractions are within walking distance!All of Vienna’s sights are within walking distance. In only a few minutes walk you will come across the largest and longest shopping streets in the city, a real paradise for shopaholics. The city centre and natural and art history museums are practically at your doorstep, while the world famous ‘Staatsoper’ is only a few minutes away. Reaching the public transport network is simple and in no other district in the city are you better connected than in the inner districts. There are a number",null,https://a0.muscache.com/pictures/fd1aed56-e888-473b-858d-8c64d6dc620f.jpg,739130,https://www.airbnb.com/users/show/739130,Roman,2011-06-24T00:00:00.000+0000,"Vienna, Austria",I can't live without hiking in the mountains. I like to travel all over the world and I love South East Asia.,within an hour,100%,100%,f,https://a0.muscache.com/im/users/739130/profile_pic/1377624953/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/739130/profile_pic/1377624953/original.jpg?aki_policy=profile_x_medium,Alsergrund,1,1,"['email', 'phone', 'work_email']",t,t,null,Alsergrund,null,48.21606,16.34875,Entire condo,Entire home/apt,3,null,1 bath,1,2,"[""Dishes and silverware"", ""Cooking basics"", ""Shampoo"", ""Long term stays allowed"", ""Cable TV"", ""Wifi"", ""Elevator"", ""Hangers"", ""Paid dryer \u2013 In building"", ""Stove"", ""Hot water"", ""Hot water kettle"", ""Iron"", ""Essentials"", ""Dishwasher"", ""Free washer \u2013 In building"", ""Oven"", ""Coffee maker"", ""Hair dryer"", ""1\"" TV with standard cable, DVD player"", ""Bed linens"", ""Bathtub"", ""Microwave"", ""Paid parking off premises"", ""Refrigerator"", ""Carbon monoxide alarm"", ""Kitchen"", ""Room-darkening shades"", ""Extra pillows and blankets"", ""Pocket wifi"", ""Host greets you"", ""Smoke alarm"", ""Heating""]",$52.00,1,182,1,5,1125,1125,4.8,1125.0,null,t,5,11,11,233,2022-09-12T00:00:00.000+0000,144,16,2,2011-08-19T00:00:00.000+0000,2022-08-24T00:00:00.000+0000,4.47,4.56,4.7,4.69,4.64,4.66,4.59,null,t,1,1,0,0,1.07
197968,https://www.airbnb.com/rooms/197968,20220911230927,

Now that we're happy with our Delta Table, we can clean up our directory using `VACUUM`. Vacuum accepts a retention period in hours as an input.

In [0]:
#from delta.tables import *

#deltaTable = DeltaTable.forPath(spark, deltaPath)
#deltaTable.vacuum(0)

Uh-oh, our code doesn't run! By default, to prevent accidentally vacuuming recent commits, Delta Lake will not let users vacuum a period under 7 days or 168 hours. Once vacuumed, you cannot return to a prior commit through time travel, only your most recent Delta Table will be saved.

We can workaround this by setting a spark configuration that will bypass the default retention period check.

In [0]:
from delta.tables import *

spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
deltaTable = DeltaTable.forPath(spark, deltaPath)
deltaTable.vacuum(0)

Out[40]: DataFrame[]

Let's take a look at our Delta Table files now. After vacuuming, the directory only holds the partition of our most recent Delta Table commit.

In [0]:
display(dbutils.fs.ls(deltaPath + "/neighbourhood_cleansed=Favoriten/"))

path,name,size,modificationTime
dbfs:/tmp/renato/neighbourhood_cleansed=Favoriten/part-00000-3996de0e-e0e7-48e3-ad58-27facdb07da9.c000.snappy.parquet,part-00000-3996de0e-e0e7-48e3-ad58-27facdb07da9.c000.snappy.parquet,177668,1666269104000


Since vacuuming deletes files referenced by the Delta Table, we can no longer access past versions. The code below should throw an error.

In [0]:
#df = spark.read.format("delta").option("versionAsOf", 0).load(deltaPath)
#display(df)

## Deleting files and tables created

#### Deleting Delta Table on Default database

In [0]:
%sql
USE dsacademy_embedded_wave3_$username;
DROP TABLE deltareview;

#### Deleting Delta Table on folder

In [0]:
%sql
DROP TABLE IF EXISTS DELTA.`/tmp/renato/`;

#### Deleting Files

In [0]:
import shutil
shutil.rmtree(deltaPath)

Code modified and enhanced from 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>